Dhananjay Tiwari,
Ph.D. Mechanical Engineering,
Mechanical Engineering Laboratory, 1204
University of Illinois at Urbana Champaign

This file has the code for implementation of FLPO optimization problem using using scipy optimizers

In [4]:
# include libraries and dependencies
import numpy as np # eh! computational python programming and not include this? Can't happen!

In [5]:
# define the structure of FLPO problem
N = 1 # number of nodes
M = 3 # number of facilities


In [35]:
# create FLPO class and define its associated functions
class FLPO():

    # define data members of the class
    N : int # number of nodes
    M : int # number of facilities
    K : int # number of stages
    node_id : list # node indices
    fac_id : list # facility indices
    dest_id : list # destination id
    fac_param : dict # facility parameters

    stages : list # FLPO stages, K elements
                  #   Nx1  (M+1)x1  (M+1)x1   (M+1)x1    (M+1)x1       1x1
                  #   stg0   stg1    stg2     stg3 . . . . stgK-2     stgK-1
                  #    0      0        0        0  . . . .  0               
                  #    1      1        1        1  . . . .  1
                  #    2      2        2        2  . . . .  2          dest
                  #    .      .        .        .  . . . .  .
                  #    .      .        .        .  . . . .  .
                  #    .      M-1      M-1      M-1  . . .  M-1
                  #    .      dest    dest     dest  . . .  dest
                  #    N-1    

    def __init__(self, N, M):
        # this function intializes the FLPO structure with assigned values
        
        # nodes, facilities and destination indices
        self.N = N # initialize the number of nodes
        self.M = M # initialize the number of facilities
        self.K = self.M + 2 # 1 initial stage + #intermediate stages equal to the number of facilities + 1 final stage
        self.node_id = list(range(N)) # 0 1 2 . . . N-1
        self.fac_id = list(range(N, N+M)) # N N+1 . . . N+M-1
        self.dest_id = list(range(N+M, N+M+1)) # N+M
        
        # initialize parameters associated to all the facilities
        self.fac_param = dict()
        # self.fac_param['locations'] = np.zeros([M, 2])
        self.fac_param['schedules'] = np.zeros([M, N])

        # define the structure of the stages and the elements in them
        self.stages = [0 for element in range(self.K)] # initialize list stages. Each 0 to be replaced with a column vector
        for stg in range(self.K): # i runs from 0 to M+1
            if stg == 0:
                # initial stage consisting of nodes
                self.stages[stg] = self.node_id # Nx1 array
            elif stg == self.K-1:
                # final stage consists of only destination
                self.stages[stg] = self.dest_id # 1x1 array
            else:
                # other stages consist of facilities and destination
                self.stages[stg] = self.fac_id + self.dest_id # (M+1) x 1 array
        
        # initialize association probabilities
        self.asn_pb = [[0 for element1 in range(self.K-1)] for element2 in range(N)]
        for n in range(N): # iteration over each node in the initial stage
            for stg in range(self.K-1): # iteration goes until the penultimate stage
                next_stage = self.stages[stg+1] # get the next stage nodes
                # initialize probabilities as ones (which is not valid probability distribution)
                self.asn_pb[n][stg] = np.ones([len(self.stages[stg]), len(self.stages[stg+1])])
                # hence normalize the probability associations
                for i in self.stages[stg]:
                    i = i - self.stages[stg][0] # first element in a stage is not necessarily indexed zero
                    self.asn_pb[n][stg][i, :] = np.ones(len(next_stage))/len(next_stage) # normalize probabilities

    # define stage transportation cost for each node        
    def cost_stage_transit(self, n, stg, i, j):
        # this function gives the value of stage transition cost for node n
        # from node i in stage stg to node j in stage stg+1
        # for now the cost is defined based on the facility schedules

        t = self.fac_param('schedules') # a 2D matrix with each row representing a facility, and every column representing a node
        # shift the stage indices
        i = i - self.stages[stg][0]
        j = j - self.stages[stg][0]
        cost = t[j, n] - t[i, n] # time taken to move from node i to node j by vehicle n
        return cost

    # define the distortion function of a node
    def distortion(self, n): # depends on probability associations and parameters of facilities
        # this function computes the distortion function of a node n

        # initialize the distortion data structure
        D = [np.zeros(len(self.stages[stg])) for stg in range(K)]

        # iterate over stages starting from the final stage to the initial stage
        stage_indices = list(range(len(self.stages)))
        reversed_stage_indices = stage_indices[::-1]
        for stg in reversed_stage_indices:
            # iterate over each element of every stage
            for i in self.stages[stg]:
                i = i - self.stages[stg][0]
                D[stg][i] = 0
                if stg == self.K-1:
                    D[stg][i] = 0
                else:
                    for j in self.stages[stg+1]:
                        D[stg][i] = D[stg][i] + self.asn_pb[n][stg][i, j-self.stages[stg+1][0]]* \
                            (D[stg][j-self.stages[stg+1][0]] + self.cost_stage_transit(n, stg, i, j))
        return D
    

    def entropy(pb_asn): # depends on probability associations
        return 0
    
    def free_energy():
        return 0



Test the FLPO class

In [37]:
# create an instance of FLPO class
N = 2
M = 3
flpo = FLPO(N, M)
stages = flpo.stages
associations = flpo.asn_pb
# facility_locations = flpo.fac_param['locations']
facility_schedules = flpo.fac_param['schedules']
print(stages)
print(associations)
# print(facility_locations)
print(facility_schedules)
D = [np.zeros(len(flpo.stages[stg])) for stg in range(flpo.K)]
print(D)

[[0, 1], [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5], [5]]
[[array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[1.],
       [1.],
       [1.],
       [1.]])], [array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[1.],
       [1.],
       [1.],
       [1.]])]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[array([0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0.])]
